# Prepare Environment

In [ ]:
from os import path
from utils.notebook_helpers import select_device, show_in_netron
from models.cnv import CNV

data_dir = path.join('..', 'data')
model_name = 'CNV'
model_dir = 'exports'
model_path = path.join(model_dir, f'{model_name}.pth')
device = select_device()

num_classes = 2 # noperson, person
weight_bit_width = 1
act_bit_width = 1
in_bit_width = 8
in_ch = 3 # for each image color channel, RGB

print('Device:', device)


To ensure reproducible results, configure determinism for applicable libraries.

We were not yet able to make this training fully deterministic, likely due to library code that does not respect the set seed.
But it seems like this gives us comparably close results, because at least some parts are working deterministically (as intended).

In [ ]:
from utils.notebook_helpers import set_determinism

seed = 0
set_determinism(seed)


# Load Dataset

See the Docstring of `CustomDataSet` for the expected directory structure.

In [ ]:
from datasets.custom import CustomDataSet

dataset = CustomDataSet(
    root_dir=data_dir,
    dataset='custom_dataset',
    validate_ratio=0.2,
    image_size=64,
    batch_size=32,
    num_workers=0,
    seed=seed
)


In [ ]:
from utils.notebook_helpers import print_num_samples

print_num_samples("train", dataset.train_loader.dataset, dataset.classes)
print_num_samples("validate", dataset.validate_loader.dataset, dataset.classes)
print_num_samples("test", dataset.test_loader.dataset, dataset.classes)


# Show Class Ids

In [ ]:
dataset.classes


# Training

In [ ]:
from trainers.basic import Trainer

model = CNV(num_classes, weight_bit_width, act_bit_width, in_bit_width, in_ch)
trainer = Trainer(
    model=model,
    save_path=model_path,
    train_dataloader=dataset.train_loader,
    validate_dataloader=dataset.validate_loader,
    device=device,
    max_epochs=10,
    patience=5,
    print_every_n_batches=10
)
trainer.train()


In [ ]:
trainer.plot_history()


## Measure accuracy

In [ ]:
from testers.basic import Tester

model = CNV(num_classes, weight_bit_width, act_bit_width, in_bit_width, in_ch)
Tester(model=model, load_path=model_path, test_dataloader=dataset.test_loader, device=device, print_test_information=True).test()


## FINN Export

In [ ]:
from exporters.onnx import ONNXExporter

model = CNV(num_classes, weight_bit_width, act_bit_width, in_bit_width, in_ch)
onnx_path = path.join(model_dir, f'{model_name}.onnx')
ONNXExporter(model=model, load_path=model_path, save_path=onnx_path).export()

show_in_netron(onnx_path)
